In [ ]:
# Copyright (c) 2022 The MATCH Authors. All rights reserved.
# Licensed under the GNU AFFERO GENERAL PUBLIC LICENSE Version 3 (or later), which is in the LICENSE file.

from msilib.schema import File
import os
import shutil
import pandas as pd


## Specify the filepath of the model run folder

In [ ]:
model_run_folder = '../../MODEL_RUNS/test'

## Run summary reports for all scenarios
This is useful if you want to re-run all of the summary reports for a completed model run using an updated summary report template. This will also generate the scenario comparison table.

In [ ]:
# get a list of all scenarios
scenarios = os.listdir(f"{model_run_folder}/outputs")

for scenario in scenarios:
    inputs_dir = f"{model_run_folder}/inputs/{scenario}"
    outdir = f"{model_run_folder}/outputs/{scenario}"
    print(scenario)

    shutil.copy("../reporting/summary_report.ipynb", inputs_dir)

    # run the notebook
    os.system(
        f'jupyter nbconvert --ExecutePreprocessor.kernel_name="python3" --to notebook --execute --inplace {inputs_dir}/summary_report.ipynb'
    )
    # convert the notebook to html and save it to the output directory
    os.system(
        f"jupyter nbconvert --to html --no-input --no-prompt {inputs_dir}/summary_report.ipynb --output-dir {outdir}/../../summary_reports --output summary_report_{scenario}"
    )
    # delete the notebook from the inputs directory
    os.remove(f"{inputs_dir}/summary_report.ipynb")


i = 0
for s in scenarios:
    summary_file = f"{model_run_folder}/summary_reports/scenario_summary_{s}.csv"
    capacity_file = f"{model_run_folder}/outputs/{s}/gen_cap.csv"
    predetermined = f"{model_run_folder}/inputs/{s}/gen_build_predetermined.csv"
    predetermined_list = list(
        pd.read_csv(predetermined, usecols=["GENERATION_PROJECT"])["GENERATION_PROJECT"]
    )

    if i == 0:
        summary_df = pd.read_csv(summary_file, index_col=0)
        capacity_df = pd.read_csv(
            capacity_file, usecols=["generation_project", "gen_tech", "GenCapacity"]
        ).rename(columns={"GenCapacity": s})
        capacity_df["predetermined"] = "No"
        capacity_df.loc[
            capacity_df["generation_project"].isin(predetermined_list), "predetermined"
        ] = "Yes"

        i += 1
    else:
        try:
            summary_df2 = pd.read_csv(summary_file, index_col=0)
            summary_df = summary_df.merge(
                summary_df2, how="outer", left_index=True, right_index=True
            )

            capacity_df2 = pd.read_csv(
                capacity_file, usecols=["generation_project", "gen_tech", "GenCapacity"]
            ).rename(columns={"GenCapacity": s})
            capacity_df2["predetermined"] = "No"
            capacity_df2.loc[
                capacity_df2["generation_project"].isin(predetermined_list),
                "predetermined",
            ] = "Yes"

            capacity_df = capacity_df.merge(
                capacity_df2,
                how="outer",
                on=["generation_project", "gen_tech", "predetermined"],
            )
        except FileNotFoundError:
            pass

summary_df.columns = summary_df.loc["Scenario Name", :]
summary_df = summary_df.drop(index="Scenario Name")
summary_df.to_csv(f"{model_run_folder}/summary_reports/scenario_comparison.csv")
capacity_df.to_csv(
    f"{model_run_folder}/summary_reports/portfolio_comparison.csv", index=False
)



# Run the summary report for a single scenario
This is useful to test run a single summary report

In [ ]:
# Specify the name of the scenario
#############################################
scenario = "Name"
#############################################


inputs_dir = f"{model_run_folder}/inputs/{scenario}"
outdir = f"{model_run_folder}/outputs/{scenario}"
print(scenario)

shutil.copy("../reporting/summary_report.ipynb", inputs_dir)

# run the notebook
os.system(
    f'jupyter nbconvert --ExecutePreprocessor.kernel_name="python3" --to notebook --execute --inplace {inputs_dir}/summary_report.ipynb'
)
# convert the notebook to html and save it to the output directory
os.system(
    f"jupyter nbconvert --to html --no-input --no-prompt {inputs_dir}/summary_report.ipynb --output-dir {outdir}/../../summary_reports --output summary_report_{scenario}"
)
# delete the notebook from the inputs directory
os.remove(f"{inputs_dir}/summary_report.ipynb")



## Run summary reports for only the scenarios where the report didn't complete
If certain summary report notebooks did not complete while running multiple scenarios, this will identify which ones were unsuccessful and re-run them. This will also then generate the scenario comparison table

In [ ]:
# get a list of all scenarios
scenarios = os.listdir(f"{model_run_folder}/outputs")

# get a list of all scenarios that don't have a scenario summary csv
completed_scenarios = []
for file in os.listdir(f"{model_run_folder}/summary_reports"):
    if file.endswith(".csv"):
        scenario_name = file.replace("scenario_summary_", "").replace(".csv", "")
        completed_scenarios.append(scenario_name)

# get a list of all scenarios that need to be run
scenarios_to_run = [s for s in scenarios if s not in completed_scenarios]

for scenario in scenarios_to_run:
    inputs_dir = f"{model_run_folder}/inputs/{scenario}"
    outdir = f"{model_run_folder}/outputs/{scenario}"
    print(scenario)

    shutil.copy("../reporting/summary_report.ipynb", inputs_dir)

    # run the notebook
    os.system(
        f'jupyter nbconvert --ExecutePreprocessor.kernel_name="python3" --to notebook --execute --inplace {inputs_dir}/summary_report.ipynb'
    )
    # convert the notebook to html and save it to the output directory
    os.system(
        f"jupyter nbconvert --to html --no-input --no-prompt {inputs_dir}/summary_report.ipynb --output-dir {outdir}/../../summary_reports --output summary_report_{scenario}"
    )
    # delete the notebook from the inputs directory
    os.remove(f"{inputs_dir}/summary_report.ipynb")

i = 0
for s in scenarios:
    summary_file = f"{model_run_folder}/summary_reports/scenario_summary_{s}.csv"
    capacity_file = f"{model_run_folder}/outputs/{s}/gen_cap.csv"
    predetermined = f"{model_run_folder}/inputs/{s}/gen_build_predetermined.csv"
    predetermined_list = list(
        pd.read_csv(predetermined, usecols=["GENERATION_PROJECT"])["GENERATION_PROJECT"]
    )

    if i == 0:
        summary_df = pd.read_csv(summary_file, index_col=0)
        capacity_df = pd.read_csv(
            capacity_file, usecols=["generation_project", "gen_tech", "GenCapacity"]
        ).rename(columns={"GenCapacity": s})
        capacity_df["predetermined"] = "No"
        capacity_df.loc[
            capacity_df["generation_project"].isin(predetermined_list), "predetermined"
        ] = "Yes"

        i += 1
    else:
        summary_df2 = pd.read_csv(summary_file, index_col=0)
        summary_df = summary_df.merge(
            summary_df2, how="outer", left_index=True, right_index=True
        )

        capacity_df2 = pd.read_csv(
            capacity_file, usecols=["generation_project", "gen_tech", "GenCapacity"]
        ).rename(columns={"GenCapacity": s})
        capacity_df2["predetermined"] = "No"
        capacity_df2.loc[
            capacity_df2["generation_project"].isin(predetermined_list), "predetermined"
        ] = "Yes"

        capacity_df = capacity_df.merge(
            capacity_df2,
            how="outer",
            on=["generation_project", "gen_tech", "predetermined"],
        )

summary_df.columns = summary_df.loc["Scenario Name", :]
summary_df = summary_df.drop(index="Scenario Name")
summary_df.to_csv(f"{model_run_folder}/summary_reports/scenario_comparison.csv")
capacity_df.to_csv(
    f"{model_run_folder}/summary_reports/portfolio_comparison.csv", index=False
)



# Run the comparison reports

In [ ]:
# get a list of all scenarios
scenarios = os.listdir(f"{model_run_folder}/outputs")

i = 0
for s in scenarios:
    summary_file = f"{model_run_folder}/summary_reports/scenario_summary_{s}.csv"
    capacity_file = f"{model_run_folder}/outputs/{s}/gen_cap.csv"
    predetermined = f"{model_run_folder}/inputs/{s}/gen_build_predetermined.csv"
    predetermined_list = list(
        pd.read_csv(predetermined, usecols=["GENERATION_PROJECT"])["GENERATION_PROJECT"]
    )

    if i == 0:
        summary_df = pd.read_csv(summary_file, index_col=0)
        capacity_df = pd.read_csv(
            capacity_file, usecols=["generation_project", "gen_tech", "GenCapacity"]
        ).rename(columns={"GenCapacity": s})
        capacity_df["predetermined"] = "No"
        capacity_df.loc[
            capacity_df["generation_project"].isin(predetermined_list), "predetermined"
        ] = "Yes"

        i += 1
    else:
        try:
            summary_df2 = pd.read_csv(summary_file, index_col=0)
            summary_df = summary_df.merge(
                summary_df2, how="outer", left_index=True, right_index=True
            )

            capacity_df2 = pd.read_csv(
                capacity_file, usecols=["generation_project", "gen_tech", "GenCapacity"]
            ).rename(columns={"GenCapacity": s})
            capacity_df2["predetermined"] = "No"
            capacity_df2.loc[
                capacity_df2["generation_project"].isin(predetermined_list),
                "predetermined",
            ] = "Yes"

            capacity_df = capacity_df.merge(
                capacity_df2,
                how="outer",
                on=["generation_project", "gen_tech", "predetermined"],
            )
        except FileNotFoundError:
            pass

summary_df.columns = summary_df.loc["Scenario Name", :]
summary_df = summary_df.drop(index="Scenario Name")
summary_df.to_csv(f"{model_run_folder}/summary_reports/scenario_comparison.csv")
capacity_df.to_csv(
    f"{model_run_folder}/summary_reports/portfolio_comparison.csv", index=False
)

